
# Sample Bin Packing Fitness

This notebook is to figure out ways to represent the problem landscape, and write fitness functions appropriate for the representation.


[Offline bin-packing](https://en.wikipedia.org/wiki/Bin_packing_problem#Offline_algorithms) - Ability to see all items before placing them into bins and allows improved approximation ratios.


# Current Idea

## Representation

- Individuals are lists of binary strings
- Each string represents a bin, and all strings are of length $n$, the number of items
- 0 means that the item is not in current bin, and 1 means the item is in current bin
- length of list would be Number of bins
- also have an array which stores weights
  - e.g. for bin: \[10**1**01\], we can get the italicised 1's weight by getting the weight at the 2nd index in the weights array
   

## Uncertain parts

- To determine whether or not an item can be put into a bin, a function can be used
  - Function would be an approximation algorithm, such as Best Fit Decreasing
  - This would be the population generation function (I guess?)

- Not sure how to maintain constraint on individuals after crossover and/or mutation
  - A repair function most likely needs to be introduced 
  - Or implement a constraint check (how tho???)
  - Allow (discourage / punish via fitness function)
  
Constraint and repair function may introduce bias towards a local optima instead of global depending on the problem landscape and degree of uncertainty?

Discourage / punish solutions that violates the bin capacity constraint is a more general method that does not make assumptions of problem landscape

### Recommended Testing bits

- Change algorithm so that it can take a repair function, and compare performance with non-repair counterpart


### Miscellaneous bits

- Lower bound or higher bound bins? how could they be used for the fitness function? current fitness function calculates the ratio between solution_num_bins and optimal bins (lower bound)
- Less heuristics could be a way to look at it
- Try to implement it 

### Aoife's Notes

Mutation swapping seems like the only way to do mutation - as in, the nature of the problem means we can't mutate without swapping.
E.G. if n = 4 and two of our bins are: \[1100\] and \[0011\], we want to mutate only one chromosome. The first chromosome gets selected, so we flip the bit. Then we would have \[0100\] and \[0011\], but now the first item is not in a bin. We either must create an extra bin to place the first item (which seems crazy but I guess we could try), or we flip the corresponding bit in bin 2, so that the item is now in the other bin.

The only difference between this basic example and our implementation is that our implementation will have many bins, and so when we want to swap a bit, we should randomly select the bin for it to be swapped to.




# Part B



#### Read Problem File


In [13]:
# open binpacking.txt

with open("Binpacking.txt", "r") as file:
    lines = file.readlines()


for _ in range(17):
    # removes till 'BPP      1'
    lines.pop(0)

# from here, text file repeats in pattern
# BPP      i
# number of different weights
# capacity of bin
# weight          number of items (repeats number of different weights times)

num_weights = []
capacities = []
item_weights = []
weights = []

# loop through 5 BPP scenarios
for i in range(5):
    # remove seperator BPP i
    lines.pop(0)

    # saves number of different weights
    # and capacity of bin
    num_weights.append(int(lines.pop(0)))
    capacities.append(int(lines.pop(0)))

    # make new map for BPP scenario i
    item_weights.append({}) # append []
    # loop through number of different weights
    for j in range(num_weights[i]):
        # split line into weight and number of items with weight
        line = lines.pop(0).split()

        #TODO: Change so this section reading in weights to a list
        for _ in range(int(line[1])):
            weights.append(int(line[0]))

        # map weight to number of items with weight
        # item_weights[i][int(line[0])] = int(line[1])

# close file, idk if this is needed
file.close()

# debug print for sanity check
# cuz im going insane
print(item_weights)
print(weights)
print(capacities)
print(num_weights)


[{}, {}, {}, {}, {}]
[200, 200, 200, 199, 198, 198, 197, 197, 194, 194, 193, 192, 191, 191, 191, 190, 190, 189, 188, 188, 187, 187, 186, 185, 185, 185, 185, 184, 184, 184, 183, 183, 183, 182, 182, 182, 181, 181, 180, 179, 179, 179, 179, 178, 177, 177, 177, 177, 175, 174, 173, 173, 172, 171, 171, 171, 170, 170, 169, 169, 169, 167, 167, 165, 165, 164, 163, 163, 163, 163, 162, 161, 160, 160, 159, 158, 158, 158, 157, 156, 156, 156, 156, 156, 156, 155, 155, 155, 154, 154, 153, 152, 152, 152, 151, 151, 150, 150, 150, 150, 200, 200, 199, 199, 199, 199, 198, 197, 196, 196, 195, 195, 194, 194, 193, 191, 191, 190, 189, 189, 188, 187, 187, 186, 185, 185, 184, 184, 184, 184, 184, 183, 182, 181, 181, 181, 180, 180, 179, 179, 178, 176, 175, 175, 174, 174, 174, 174, 174, 173, 172, 172, 172, 171, 170, 170, 170, 170, 169, 169, 168, 167, 167, 167, 167, 167, 165, 165, 164, 164, 163, 163, 163, 162, 162, 160, 160, 159, 159, 158, 158, 157, 157, 157, 157, 156, 156, 156, 155, 155, 154, 153, 153, 153, 152, 152


### Fitness Function and optimum bin amount calc


In [5]:
import matplotlib.pyplot as plt
import random
import math

MUTATION_RATE = 0.1
CROSSOVER_RATE = 0.9
NUM_GENERATIONS = 1000
ELITE_FACTOR = 1/10


In [6]:

# weights is a map of weights to number of items with that weight
# calculate optimal number of bins

def optimal_bin_count(weights, capacity):
    sum_weight = 0
    for weight in weights:
        sum_weight += weight * weights[weight]
    return math.ceil(sum_weight / capacity)

# bin-packing fitness function
# calculates fitness by dividing optimal number of bins by number of bins used
# and multiplying by 100 for percentage

def bpp_bin_ratio_fitness(optimal_bins, bins):
    return bins / optimal_bins * 100

# bin-packing fitness function alternative
# calculates fitness by determining error for each bin
# actual bin weight - capacity
# perfect bin = 0

def bpp_capacity_error_fitness(bin, item_weights, capacity):
    sum_weight = 0
    for i in range(len(bin)):
        if bin[i] == 1:
            sum_weight += item_weights[i]
    error = (sum_weight - capacity) ** 2
    return error

In [7]:

# Tournament selection function

# selects k individuals from the population at random
# returns the index of the best individual

def tournament_selection(population, scores, k=3):
    # Select k individuals from population at random
    selection_i = random.randrange(len(population))
    for i in [random.randrange(len(population)) for _ in range(k-1)]:
        # Check if better (e.g. perform a tournament)
        if scores[i] < scores[selection_i]:
            selection_i = i
    # Return the index of the best
    return population[selection_i]


In [8]:

# mutation function
# each item has a 1/n chance of being mutated
# if mutated, item is swapped with another item

def mutate(bins, bin, mutation_rate):
    for i in range(len(bin)):
        if random.random() < mutation_rate:
            for j in range(len(bin)):
                # I want to swap the item in the bin with a corresponding item from another bin
                # flip the current bit in the string
                
    return bin


In [9]:

# crossover function


In [10]:

# elitism function

def elite_select(population, scores, elite_factor=ELITE_FACTOR):
    elite_size = int(len(population) * elite_factor)
    # Sort the population based on scores in ascending order
    sorted_population = [x for _, x in sorted(zip(scores, population))]

    # Select the elite individuals
    elite = sorted_population[:elite_size]

    # Return the elite individuals
    return elite



### Algorithm


In [11]:

# change algorithm so that crossover is only performed if the function is passed in

# Generic GA algorithm that is passed different functions

def algorithm(fitness, generate_population, mutation, size, generations, pop_size, cross_rate, mutate_rate, elite_factor, crossover=None, converge_gen=100):

    # define average fitness list to plot
    avg_fitness = []

    # initial random population
    population = [generate_population(size) for _ in range(pop_size)]
    # debug - print size of population
    print("Population size: %d" % len(population))
    # debug - print length of first individual
    print("Individual length: %d" % len(population[0]))

    # Store temp best solution
    best_solution, best_fitness = 0, fitness(population[0])

    # Loop generations
    for gen in range(generations):
        # Evaluate all individuals in population
        scores = [fitness(c) for c in population]
        # calculate average fitness of generation
        avg_fitness.append(sum(scores) / pop_size)

        # if there are enough generations to check for convergence
        if gen >= converge_gen:
            last_n_avg_fitnesses = avg_fitness[gen-converge_gen:gen]
            # exit on convergence
            if last_n_avg_fitnesses.count(last_n_avg_fitnesses[0]) == converge_gen:
                print("Converged at generation %d" % gen)
                break

        # print best score every 100 generations
        if gen % 100 == 0:
            print("Generation %d, best score = %.3f, average fitness = %.3f" %
                  (gen, min(scores), avg_fitness[gen]))

        # Check for new best solution
        for i in range(pop_size):
            if scores[i] < best_fitness:
                best_solution, best_fitness = population[i], scores[i]
                print("Generation %d, new best f(%s) = %.3f" %
                      (gen, population[i], scores[i]))

        # Select parents to crossover
        parents = [tournament_selection(population, scores)
                   for _ in range(pop_size)]
        # Create next generation
        children = list()
        children.extend(elite_select(population, scores, elite_factor))

        for i in range(0, pop_size, 2):
            # Get pair of parents
            parent1, parent2 = parents[i], parents[i+1]
            # Perform crossover
            for child in crossover(parent1, parent2, cross_rate):
                # Mutate child
                mutation(child, mutate_rate)
                # Add to next generation
                children.append(child)
        # Replace population
        population = children

    # plot average fitness over generations
    plt.plot(avg_fitness)
    plt.xlabel('Generation')
    plt.ylabel('Average Fitness')
    plt.show()

    return [best_solution, best_fitness]

In [12]:
if __name__ == '__main__':
    print("BPP - 1")
    print("Running algorithm...")
    # TODO: write population generation function
    solution, score = algorithm(
        bpp_fitness, binary_string_generator, STRING_SIZE, NUM_GENERATIONS, POPULATION_SIZE, CROSSOVER_RATE, MUTATION_RATE, ELITE_FACTOR)
        
    print("Completed operations")

BPP - 1
Running algorithm...


NameError: name 'bpp_fitness' is not defined